In [1]:
from configparser import ConfigParser
from datetime import datetime
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import kstest
from scipy import stats as s
import statistics
import pyodbc

In [2]:
# CHANGE THE DIRECTORY TO NOTEBOOK/NOTEBOOK
try:
    cd /notebooks/notebooks
except:
    %cd /notebooks/notebooks

/notebooks/notebooks


In [3]:
CurrentDate = str(datetime.now())

delimiter1 = " ";
delimiter2 = "-";
CurrentDate = CurrentDate.replace(delimiter2, delimiter1);
CD = CurrentDate.split(delimiter1);
date = CD[2]
month = CD[1]
year = CD[0]

In [4]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
# conn.autocommit = False
cursor = conn.cursor()

In [5]:
# CHECK IF JOB IS RUNNING FOR THE FIRST TIME.
count_of_audit_ingt = pd.read_sql_query("select count(1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN where JOB_NAME = 'REPORT_RISK' and JOB_STATUS = 'Success'",conn)

In [6]:
count_of_audit_ingt = count_of_audit_ingt.iloc[0,0]

In [7]:
try:
    
    
#     PEER GROUPING TO BE DONE IF DATE MATCHES OR ITS FIRST RUN.
    if ((date == '17' and month == '05') or ((date == '01' and month == '05')) or (count_of_audit_ingt == 0)):
        

        
        sql_delete_query1  = "delete from FINCORE_DB.Fincore_Analytics.RISK_REPORT"
        sql_delete_query2  = "delete from FINCORE_DB.Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'R0%'"
        sql_delete_query3  = "delete from FINCORE_DB.Fincore_Analytics.RISK_REPORT_RULES"
        sql_delete_query4  = "delete from FINCORE_DB.Fincore_Analytics.RISK_REPORT_SUMMARY_STG"
        sql_delete_query5  = "delete from FINCORE_DB.Fincore_Analytics.RISK_REPORT_INCREMENTAL"
        
        cursor.execute(sql_delete_query1)
        cursor.execute(sql_delete_query2)
        cursor.execute(sql_delete_query3)
        cursor.execute(sql_delete_query4)
        cursor.execute(sql_delete_query5)
        conn.commit()

        %run Report_Risk/REPORT_RISK_CR.ipynb
        %run Report_Risk/REPORT_RISK_2.ipynb
        
        
    else:
        print("Else")
        %run Report_Risk/REPORT_RISK_INCREMENTAL_CR.ipynb
        %run Report_Risk/REPORT_RISK_2_INCREMENTAL.ipynb

        

    
    
    
except Exception as error:
#     IF THRE IS ERROR IN CODE , THEN UPDATE THE AUDIT INTGN.
    config = ConfigParser()
    config.read(r'/notebooks/notebooks/config.ini')
    connection_string = config.get('SERVER', 'connection')
    conn = pyodbc.connect(connection_string)
    # conn.autocommit = False
    cursor = conn.cursor()

    %store -r job_id
    error = str(error)
    print("Here error")
    error = error.replace('\'','')
    error = error.replace('\"','')
    print("Failed to update record to database rollback: {}".format(error))
    conn.rollback()
    conn.execute(f'''update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  JOB_RUN_ID = {job_id}''')
    conn.commit()
    conn.close()

/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Stored 'job_id' (int64)
Stored 'no_of_records' (int64)
('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH (HOLDLOCK) AS '
 'TARGET USING           (   SELECT    DISTINCT RTSSE.BATCH_ID,     '
 "RTSSE.REPORT_ID,       'R005001000' AS RULE_NO,   "
 'RPSS.ORGANIZATION_MASTER_ID AS ENTITY_ID ,     10 AS RISK_SCORE_STATIC,     '
 'GETDATE() AS CREATE_DATE , ENTITY_TYPE,PEER_GROUP_ID  FROM '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG RTSSE WITH '
 '(NOLOCK)   INNER JOIN '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_ORGANIZATION_SUMMARY_STG_RULES RPSS  WITH '
 '(NOLOCK)  ON RTSSE.ENTITY_ID=RPSS.ORGANIZATION_MASTER_ID   WHERE   '
 "RTSSE.ENTITY_TYPE = 'O'     AND RPSS.RULE_NO IN (  SELECT RULE_NO FROM "
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES  WITH (NOLOCK)  WHERE      '
 "RISK_TYPE='AUTO HIGH RULES ')       UNION     SELECT    DISTINCT            "
 "RTSSE.BATCH_ID,      RTSSE.REPORT_ID,        'R005001000' AS RULE_NO,        "
 'RPSS.PERSON_MASTER_ID AS ENTITY_ID

('MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT_RULES WITH (HOLDLOCK)   '
 "AS TARGET USING (     SELECT BATCH_ID,   REPORT_ID,   'R005006000' AS "
 'RULE_NO,     COUNT(DISTINCT TRANSACTION_ID) AS RULE_VALUE,    NULL AS '
 'RISK_SCORE_STATIC,  GETDATE() AS  CREATE_DATE,  '
 'ENTITY_ID,ENTITY_TYPE,PEER_GROUP_ID  FROM      '
 'FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG     WITH '
 '(NOLOCK)   GROUP BY BATCH_ID,REPORT_ID,ENTITY_ID,ENTITY_TYPE,PEER_GROUP_ID  '
 ') AS SOURCE    ON TARGET.BATCH_ID =SOURCE.BATCH_ID  AND '
 "TARGET.REPORT_ID=SOURCE.REPORT_ID  AND TARGET.RULE_NO ='R005006000'   AND "
 'TARGET.ENTITY_ID=SOURCE.ENTITY_ID  AND TARGET.ENTITY_TYPE '
 '=SOURCE.ENTITY_TYPE       WHEN MATCHED THEN UPDATE SET         '
 'TARGET.RULE_VALUE=SOURCE.RULE_VALUE,TARGET.PEER_GROUP_ID=SOURCE.PEER_GROUP_ID      '
 'WHEN NOT MATCHED BY TARGET THEN         '
 'INSERT(BATCH_ID,REPORT_ID,RULE_NO,RULE_VALUE,RISK_SCORE_STATIC,CREATE_DATE,ENTITY_ID,ENTITY_TYPE,PEER_GROUP_ID)   '
 '

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.5 1.5 1 0.5 NA
Not applicable
Not applicable
Not applicable
not_normal_outliers_removal
compute_IQR
outliers_right 1890041163968.75
thresh
704564479494 1063553220485 1422541961477 1781530702468 2140519443459 2499508184451 2858496925442 3217485666433 3576474407425 3935463148416 345575738502.7551 75400960002.0 758000000000 358988740991.3404 1890041163968.75
not_normal_outliers_removal
compute_IQR
outliers_right 4.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.163265306122449 1.0 1 0.46716420988814683 4.0
normal_outliers_removal
1.0
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 1.0
not_normal_outliers_removal
compute_IQR
outliers_right 2.5
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0816326530612246 1.0 1 0.2738042421428314 2.5
Not applicable
Not applicable
Not applicable
Not applicable
thresh
3813789522 5644466118 7475142715 9305819312 11136495908 12967172504 14797849101 16628525698 18459202294 20289878890 19831129

2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
1064886 1218409 1371931 1525454 1678976 1832499 1986021 2139544 2293066 2446589 911363.75 1000000.0 1000000 153522.4883923769 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
thresh
2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 1.0 1.0 1 0.0 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0 1.5 1.0 1 0.8660254037844386 NA
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
Not applicable
thresh
1064886 1218409 1371931 1525454 1678976 1832499 1986021 2139544 2293066 2446589 911363.75 1000000.0 1000000 153522.4883923769 NA
Not applicable
thresh
3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0

181it [00:00, 961.54it/s]


UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 100712 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 102012 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 102013 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 103112 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 103216 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 103217 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 103312 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 103313 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 104014 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHERE REPORT_ID = 104112 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHE

UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 118314 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 118315 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 118316 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 118512 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 118513 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 118612 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 6 WHERE REPORT_ID = 118613 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 8 WHERE REPORT_ID = 118813 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 8 WHERE REPORT_ID = 118814 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 7 WHERE REPORT_ID = 119214 
UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT SET RISK_SCORE = 9 WHE